In [78]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 2s (197 kB/s)
Reading package lists... Done


In [79]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [80]:
# 1. Read in the AWS S3 bucket into a DataFrame. (reference from Lession 22, day 3, activity 2 from gitlab)
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df= spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header = True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [81]:
#get information on the df
df.describe()

DataFrame[summary: string, id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [82]:
#get schema for df
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [83]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')


In [84]:
df.columns

['id',
 'date',
 'date_built',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view']

In [85]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""SELECT date_built, round(avg(price),2) as avg_price
FROM home_sales
WHERE bedrooms = '4'
group by date_built
order by date_built""").show()




+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [86]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built, round(avg(price),2) as avg_price
FROM home_sales
WHERE bedrooms = '3'
AND bathrooms = '3'
group by date_built
order by date_built desc""").show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015| 288770.3|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [87]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""SELECT date_built, round(avg(price),2) as avg_price
FROM home_sales
WHERE bedrooms = '3'
AND bathrooms = '3'
AND floors = '2'
AND sqft_living >= '2000'
group by date_built
order by date_built desc""").show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [88]:
#6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM home_sales
WHERE price >= 350000
""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.3402259349822998 seconds ---


In [89]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [90]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [99]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM home_sales
WHERE price >= 350000
""").show()




print("--- %s seconds ---" % (time.time() - start_time))


+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.2719857692718506 seconds ---


In [92]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data (22-day3-activity 3)
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [93]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet("home_sales_partitioned")

In [94]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [100]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM p_home_sales
WHERE price >= 350000
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.20517611503601074 seconds ---


In [96]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [97]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached("home_sales"):
  print("a table is still cached")
else:
  print("all clear")

all clear
